In [2]:
import pandas as pd

customer_sheet = '1r3KvHllGkuvUhu-Z_gMQ3gQVVAeWkTZBkdFRW_aB_M4'
invoices_gid = '1234385958'

# Get the first day of the current month
today = pd.Timestamp.now().date()
first_day_this_month = today.replace(day=1)


In [3]:
df = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{customer_sheet}/export?format=csv')

In [4]:
df2 = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{customer_sheet}/export?format=csv&gid={invoices_gid}')
df2['Date'] = pd.to_datetime(df2['Date']).dt.date
df2.loc[:, ['Amount', 'Open Balance']] = df2[['Amount', 'Open Balance']].apply(pd.to_numeric, errors='coerce')
df2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 603 entries, 0 to 602
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              603 non-null    object 
 1   Transaction Type  603 non-null    object 
 2   Num               602 non-null    object 
 3   Customer          603 non-null    object 
 4   Due Date          603 non-null    object 
 5   Amount            603 non-null    float64
 6   Open Balance      603 non-null    float64
dtypes: float64(2), object(5)
memory usage: 33.1+ KB


In [5]:
df_customers = df[['Name','Payment Method', 'Phone Number', 'Email', 'Price', 'Type', 'Last']]
df_customers.loc[:, 'Price'] = pd.to_numeric(df_customers['Price'],errors='coerce')
df_customers[df_customers['Price']>0]
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552 entries, 0 to 551
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Name            552 non-null    object 
 1   Payment Method  532 non-null    object 
 2   Phone Number    515 non-null    object 
 3   Email           524 non-null    object 
 4   Price           516 non-null    float64
 5   Type            333 non-null    object 
 6   Last            82 non-null     object 
dtypes: float64(1), object(6)
memory usage: 30.3+ KB


In [13]:
df_pending_invoices = df2.loc[df2['Date']<first_day_this_month].copy()
df_current_invoices = df2.loc[df2['Date']>=first_day_this_month].copy()
df_pending_invoices.info()

<class 'pandas.core.frame.DataFrame'>
Index: 309 entries, 0 to 324
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              309 non-null    object 
 1   Transaction Type  309 non-null    object 
 2   Num               309 non-null    object 
 3   Customer          309 non-null    object 
 4   Due Date          309 non-null    object 
 5   Amount            309 non-null    float64
 6   Open Balance      309 non-null    float64
dtypes: float64(2), object(5)
memory usage: 19.3+ KB


In [28]:
num_of_invoices = df_pending_invoices.groupby("Customer")[["Open Balance"]].count()
num_of_invoices.sort_values(by='Open Balance', ascending=False)


,Open Balance
Customer,
Aally Moran,12
"Escalona, Arnaldo",12
"Moran, Liz",12
"Gutierrez, Alonso",9
Woodbridge Commerce Center LLC - Lance Bremer,8
...,...
"Viscaro, Doreen",1
"Werrenwrath, Eric",1
"Wilson, Pete",1


In [42]:
invoice_details = df_pending_invoices.groupby("Customer")['Num'].agg('\n '.join).reset_index()
print(invoice_details['Num'].iloc[0])


26291
 26832
 27380
 27987
 28566
 29159
 29740
 30301
 30830
 31377
 31934
 32476


In [ ]:
df_pending_invoices['Invoice Details'] = '#' + df_pending_invoices['Num'].astype(str) + ', ' + df_pending_invoices['Date'].astype(str) + ', $' + df_pending_invoices['Open Balance'].astype(str) + ' - Invoice Link'
invoice_details = df_pending_invoices.groupby("Customer")['Invoice Details'].agg('\n '.join).reset_index()
invoice_details

,Date,Transaction Type,Num,Customer,Due Date,Amount,Open Balance,Inv Details,Invoice Details
0,2024-03-01,Invoice,22879,"Acker, Mark",03/31/2024,135.0,135.0,"#22879, 2024-03-01, $135.0 - Invoice Link","#22879, 2024-03-01, $135.0 - Invoice Link"
1,2024-04-01,Invoice,20711,"Zaragoza, Beatrice",04/16/2024,120.0,120.0,"#20711, 2024-04-01, $120.0 - Invoice Link","#20711, 2024-04-01, $120.0 - Invoice Link"
2,2024-04-01,Invoice,22880,"Acker, Mark",05/01/2024,135.0,135.0,"#22880, 2024-04-01, $135.0 - Invoice Link","#22880, 2024-04-01, $135.0 - Invoice Link"
3,2024-05-01,Invoice,22881,"Acker, Mark",05/31/2024,135.0,135.0,"#22881, 2024-05-01, $135.0 - Invoice Link","#22881, 2024-05-01, $135.0 - Invoice Link"
4,2024-06-01,Invoice,22882,"Acker, Mark",07/01/2024,135.0,135.0,"#22882, 2024-06-01, $135.0 - Invoice Link","#22882, 2024-06-01, $135.0 - Invoice Link"
...,...,...,...,...,...,...,...,...,...
304,2025-12-19,Invoice,32506,"Ginzburg, Sergei 2",12/31/2025,30.0,30.0,"#32506, 2025-12-19, $30.0 - Invoice Link","#32506, 2025-12-19, $30.0 - Invoice Link"
305,2025-12-02,Invoice,32476,Aally Moran,01/01/2026,1.0,1.0,"#32476, 2025-12-02, $1.0 - Invoice Link","#32476, 2025-12-02, $1.0 - Invoice Link"
308,2025-12-18,Invoice,32508,"Hewitt, Bobby",01/07/2026,658.0,658.0,"#32508, 2025-12-18, $658.0 - Invoice Link","#32508, 2025-12-18, $658.0 - Invoice Link"
321,2025-12-10,Invoice,32487,"Moran, Liz",01/09/2026,10.0,10.0,"#32487, 2025-12-10, $10.0 - Invoice Link","#32487, 2025-12-10, $10.0 - Invoice Link"
